Thinking out loud, at the beginning:
    - Wow these files are big, I'm going to need to pare down the data probably a lot.
    - First going to be looking for "what's the most complete data that I have for what span of time"?
    - What data do I actually want to focus on? (record some research on how, broadly, one comes up with a company valuation that could be comparable to market capitalization)
    - I probably should focus my market data gathering on the day after each SEC filing is released, assuming that will be the day that stock price movements are most influenced by the filing information rather than other news or factors.
    - My core question so far is, are market prices correlated with valuation information in some fashion?
        - If so, which valuation information is most strongly correlated?
        - What, if any, is the relationship between stock price/market capitalization and the business's balance sheet?
        - Do any of these data include attempts to put a value on intangibles such as goodwill/reputation?
        - The fundamental analysis tradition focuses on ratios of price to earnings, book value, earnings growth; do the ratios that investors find acceptable change over time? Do they tend to change back (or ahead) when SEC filings are released?
        - (Do I have time for this?) What happens in the periods between filings? Do prices maintain a range, like the moving average support/resistance bands favored by technical analysis?
    - Therefore, what am I looking for in the data I've found so far?
        - As much history as I can assemble from both market data and SEC filings; will need to drop years where I have one but not the other.
        - Focus on the Fortune 100, probably choose the 100 as of the most recent year I have data for.
        - Market data at daily granularity, hopefully with open, high, low, and close prices for each day. Probably focus on dates related to the company's SEC filings.
        - Still deciding what SEC filing information, partially I don't know what I have to work with yet, hoping for a moderately quick thumbnail of value related to assets, liabilities, and maybe intangibles/goodwill. Assuming earnings-per-share numbers also will be found here.
        - If market data has already calculated P/E or other ratios, great, I can use those to sanity-check what I'm pulling in from SEC.

First crack at a plan:

Open each dataset (I have 4 right now) individually, probably not in code. Need to see file structure and years covered. Make decision on how much history to collect/collate.
    - Market datasets from:
        - https://www.kaggle.com/borismarjanovic/price-volume-data-for-all-us-stocks-etfs
            - description does not offer a begin date but last updated in 2017-Nov
            - data appears to be in txt files by ticker: file for AAPL dates to 1984-Sep and columns are Date,Open,High,Low,Close,Volume,OpenInt
            - there is a comment that prices in this data are adjusted for dividends AND splits (as opposed to Yahoo Finance data, which is not or not always--not sure what that means)
        - https://www.kaggle.com/tsaustin/us-historical-stock-prices-with-earnings-data
            - 20 years according to description; last updated 2020-Dec
            - Data in 4 files: stock price, earnings (estimated and reported), dividends, and a summary
    - SEC datasets from:
        - https://www.kaggle.com/miguelaenlle/parsed-sec-10q-filings-since-2006
            - Since 2006 according to description; Last updated 2020-Jul
            - Notes that ~50% of data is null, which might be 'not found in filing' or 'not found by parser'.
            - Does have lots of promising columns like commonstocksharesissued, assetscurrent, accountspayablecurrent, commonstockvalue, liabilities, liabilitiesandstockholdersequity, stockholdersequity, earningspersharebasic, netincomeloss, profitloss, costofgoodssold, filing_date, costsandexpenses, cash 
        - https://www.kaggle.com/finnhub/reported-financials
            - 2010-2020, according to description. Last updated 2021-Mar
            - 4 GB data!
            - Data is in folders by year-qtr; JSON files. Will take some work, I think, to find the tickers I want.
            
So, I can probably get 2010-2020 reasonably complete. Maybe 2006-2020.

Open each dataset in code and examine columns. See if I can choose one market and one SEC dataset; do they contain the same information? Is there just a column or two from one I can add to the other?

Choose 100 companies and pull their data into new files to reduce data sizes. 

Begin EDA, look for cleaning needs. 

Figure out how best to pull only the SEC-related dates of market data, assuming that dates will generally be different for each company.

Calculate (can I sanity-check somehow?) fundamental analysis ratios: P/E, P/E/G, price/book.

Other tasks as I think of them while exploring!
    - Do I have data that shows market capitalization? Or shares outstanding, so I can calculate?
    - How does one adjust prices for dividends, splits? What price do I need to use to calculate ratios, market cap, etc?

Questions for later:
    - Once I produce something with this, should I share it on Kaggle? If I've combined datasets, can I link my code to both of them?
    - Can or should I try to confirm that Yahoo Finance is a bad place to get stock data from? (see notes on first market dataset above)

Research:
https://www.investopedia.com/terms/a/adjusted_closing_price.asp
https://www.investopedia.com/articles/fundamental-analysis/08/sec-forms.asp